In [2]:
import cv2, time, threading, queue, numpy as np, yaml
import ncnn

# === 설정 ===
MODEL_PARAM = "knife01_best_50.ncnn.param"   # export 결과 파일명
MODEL_BIN   = "knife01_best_50.ncnn.bin"
IMG_SIZE    = 640                    # export에 사용한 imgsz와 일치
CONF_TH     = 0.4
IOU_TH      = 0.25
NUM_THREADS = 4                      # Pi 코어 수에 맞춰 조정(4~6)
FRAME_SKIP  = 2 
# === 클래스 이름 로드 ===
with open("pj01_data_1.yaml", "r", encoding="utf-8") as f:
    data = yaml.full_load(f)
NAMES = data.get("names", [])

# === Letterbox 전처리 ===
def letterbox(img, new=IMG_SIZE, color=(114,114,114)):
    h, w = img.shape[:2]
    r = min(new / h, new / w)
    nh, nw = int(round(h * r)), int(round(w * r))
    resized = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA)
    canvas = np.full((new, new, 3), color, dtype=np.uint8)
    top = (new - nh) // 2
    left = (new - nw) // 2
    canvas[top:top+nh, left:left+nw] = resized
    return canvas, r, left, top

# === NMS ===
def nms(dets, iou_th=IOU_TH):
    # dets: [([x1,y1,x2,y2], score, cls_id), ...] in letterbox 좌표
    if not dets: return []
    boxes = np.array([d[0] for d in dets], dtype=np.float32)
    scores = np.array([d[1] for d in dets], dtype=np.float32)
    order = scores.argsort()[::-1]
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        if order.size == 1: break
        rest = order[1:]
        xx1 = np.maximum(boxes[i,0], boxes[rest,0])
        yy1 = np.maximum(boxes[i,1], boxes[rest,1])
        xx2 = np.minimum(boxes[i,2], boxes[rest,2])
        yy2 = np.minimum(boxes[i,3], boxes[rest,3])
        inter = np.maximum(0.0, xx2 - xx1) * np.maximum(0.0, yy2 - yy1)
        area_i = (boxes[i,2] - boxes[i,0]) * (boxes[i,3] - boxes[i,1])
        area_r = (boxes[rest,2] - boxes[rest,0]) * (boxes[rest,3] - boxes[rest,1])
        iou = inter / (area_i + area_r - inter + 1e-6)
        order = rest[iou <= iou_th]
    return [dets[k] for k in keep]

# === ncnn 네트 준비 ===
net = ncnn.Net()
opt = net.opt
opt.num_threads = NUM_THREADS
opt.use_fp16_storage = True
opt.use_fp16_arithmetic = True
net.load_param(MODEL_PARAM)
net.load_model(MODEL_BIN)

# 주의: 입력/출력 blob 이름은 export 결과에 따라 다를 수 있습니다.
INPUT_BLOB  = "in0"   # 보통 'images'
OUTPUT_BLOB = "out0"  # 보통 'output0'

def infer_one(frame_bgr):
    H0, W0 = frame_bgr.shape[:2]
    # 1) 전처리(letterbox → RGB → 정규화)
    img_lbx, r, lpad, tpad = letterbox(frame_bgr, IMG_SIZE)
    img_rgb = cv2.cvtColor(img_lbx, cv2.COLOR_BGR2RGB)

    mat_in = ncnn.Mat.from_pixels(img_rgb, ncnn.Mat.PixelType.PIXEL_RGB,
                                  IMG_SIZE, IMG_SIZE)
    # YOLOv5 export 기본 전처리: 0~1 스케일(필요시 mean/std 조정)
    mat_in.substract_mean_normalize([0,0,0],
                                    [1/255.0, 1/255.0, 1/255.0])

    # 2) 추론
    ex = net.create_extractor()
    ex.input(INPUT_BLOB, mat_in)
    ret, mat_out = ex.extract(OUTPUT_BLOB)
    
    if ret != 0:
        # 출력 blob 이름이 다르면 여기서 실패합니다.
        raise RuntimeError(f"ncnn extract 실패 (code={ret}). "
                           f"INPUT={INPUT_BLOB}, OUTPUT={OUTPUT_BLOB} 확인 필요")
    
    arr = mat_out.numpy()  # ncnn.Mat -> numpy

    dets = []
    
    # out = np.array(mat_out)  # shape: (num, C)  보통 C=85: [cx,cy,w,h,conf,cls...]
    # dets = []
    # for p in out:
    #     obj_conf = float(p[4])
    #     if obj_conf < CONF_TH:
    #         continue
    #     cls_scores = p[5:]
    #     cls_id = int(np.argmax(cls_scores))
    #     cls_conf = float(cls_scores[cls_id])
    #     score = obj_conf * cls_conf
    #     if score < CONF_TH:
    #         continue

    #     cx, cy, w, h = map(float, p[:4])
    #     x1 = cx - w/2
    #     y1 = cy - h/2
    #     x2 = cx + w/2
    #     y2 = cy + h/2
    #     dets.append(([x1,y1,x2,y2], score, cls_id))
    if arr.ndim == 2 and arr.shape[0] in (6, 7, 8):
        # 당신 상황: (7,2100)이고, 0~3: cx,cy,w,h / 4~6: obj*cls0..2
        D, N = arr.shape
        A = arr.T  # (N, D) : 한 행이 한 후보
        cx, cy, w, h = A[:, 0], A[:, 1], A[:, 2], A[:, 3]

        if D == 7:
            # obj*cls 가 이미 곱해진 3개 클래스 점수
            cls_scores = A[:, 4:7]              # (N,3)
            cls_ids = np.argmax(cls_scores, axis=1)
            scores  = cls_scores[np.arange(N), cls_ids]
        elif D == 8:
            # [cx,cy,w,h,obj,cls0,cls1,cls2] (예시)
            obj = A[:, 4]
            cls_logits = A[:, 5:8]
            # Sigmoid 여부는 모델에 따라 다름. 필요하면 시그모이드 적용:
            # cls_probs = 1.0 / (1.0 + np.exp(-cls_logits))
            # 여기서는 이미 확률이라 가정
            cls_probs = cls_logits
            cls_ids = np.argmax(cls_probs, axis=1)
            cls_confs = cls_probs[np.arange(N), cls_ids]
            scores = obj * cls_confs
        else:
            # (6,N) 등 애매한 경우: 구조 확인 필요
            return []

        # 점수 필터
        keep = scores >= CONF_TH
        if not np.any(keep):
            return []

        cx = cx[keep]; cy = cy[keep]; w = w[keep]; h = h[keep]
        scores = scores[keep]; cls_ids = cls_ids[keep]

        # cxcywh -> xyxy (레터박스된 IMG_SIZE 기준)
        x1 = cx - w/2; y1 = cy - h/2
        x2 = cx + w/2; y2 = cy + h/2

        # 원본 좌표로 역변환
        x1 = (x1 - lpad) / r; y1 = (y1 - tpad) / r
        x2 = (x2 - lpad) / r; y2 = (y2 - tpad) / r

        # 클리핑
        x1 = np.clip(x1, 0, W0); y1 = np.clip(y1, 0, H0)
        x2 = np.clip(x2, 0, W0); y2 = np.clip(y2, 0, H0)

        dets = [([int(x1[i]), int(y1[i]), int(x2[i]), int(y2[i])],
                 float(scores[i]), int(cls_ids[i]))
                for i in range(len(scores))]

    else:
        # 예상 밖 출력 형식
        raise ValueError(f"알 수 없는 출력 shape: {arr.shape}")
    # 3) NMS
    dets = nms(dets, IOU_TH)

    return dets

def main():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,  640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)
    
    q = queue.Queue(maxsize=2)
    def reader():
        while True:
            ok, f = cap.read()
            if not ok: break
            if not q.full(): q.put(f)
    
    th = threading.Thread(target=reader, daemon=True); th.start()
    
    last_dets = []      # 마지막 검출 결과(스킵 프레임에 재사용)
    fid = 0
    t0 = time.time()
    
    while True:
        if q.empty():
            if cv2.waitKey(1) == 27: break
            continue
    
        frame = q.get()
        fid += 1
    
        # ===== 프레임 스킵 핵심 =====
        if fid % (FRAME_SKIP + 1) == 1:
            # 이 프레임에서만 추론 수행
            dets = infer_one(frame)
        
        # 그리기
        for (x1,y1,x2,y2), s, c in dets:
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,255), 2)
            name = NAMES[c] if 0 <= c < len(NAMES) else str(c)
            cv2.putText(frame, f"{name}:{s:.2f}", (x1, y1-6),cv2.FONT_HERSHEY_PLAIN, 1.5, (0,0,255), 2)

        # FPS
        now = time.time()
        fps = 1.0 / (now - t0)
        t0 = now
        cv2.putText(frame, f"FPS {fps:.1f}", (8,20),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

        cv2.imshow("obj(ncnn)", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'pj01_data_1.yaml'